In [3]:
# Getting the dataset
!wget https://www.dropbox.com/s/utg59on1du4ushp/Dataset.zip?dl=0

--2020-07-09 01:03:33--  https://www.dropbox.com/s/utg59on1du4ushp/Dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/utg59on1du4ushp/Dataset.zip [following]
--2020-07-09 01:03:34--  https://www.dropbox.com/s/raw/utg59on1du4ushp/Dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4641042746de16c6f8cd3aac1f.dl.dropboxusercontent.com/cd/0/inline/A7LgOw0xkoio0Sz7ZZtZelwJgIXW8za7yTaZkQzFe0XN_zq8TZcPBsPM4_ZpdKTYhOXitPU7QGQeUqcO1Ni8hPuosSlReT7emSWlb72fRqS1up6NLUbRC0NkR1m9HHhSo4k/file# [following]
--2020-07-09 01:03:34--  https://uc4641042746de16c6f8cd3aac1f.dl.dropboxusercontent.com/cd/0/inline/A7LgOw0xkoio0Sz7ZZtZelwJgIXW8za7yTaZkQzFe0XN_zq8TZcPBsPM4_ZpdKTYhOXitPU7QGQeUqcO1Ni8hPuosSlReT7emSWlb72fRqS1u

In [4]:
# Unzip the dataset
!unzip /content/Dataset.zip?dl=0

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Train/Spa/images318 (3).jpg  
  inflating: Dataset/Train/Spa/images318 (4).jpg  
  inflating: Dataset/Train/Spa/images318.jpg  
  inflating: Dataset/Train/Spa/images319 (2).jpg  
  inflating: Dataset/Train/Spa/images319 (3).jpg  
  inflating: Dataset/Train/Spa/images319 (4).jpg  
  inflating: Dataset/Train/Spa/images319.jpg  
  inflating: Dataset/Train/Spa/images32 (4).jpg  
  inflating: Dataset/Train/Spa/images32.jpg  
  inflating: Dataset/Train/Spa/images320 (3).jpg  
  inflating: Dataset/Train/Spa/images320 (4).jpg  
  inflating: Dataset/Train/Spa/images321 (2).jpg  
  inflating: Dataset/Train/Spa/images321 (3).jpg  
  inflating: Dataset/Train/Spa/images321 (4).jpg  
  inflating: Dataset/Train/Spa/images322 (2).jpg  
  inflating: Dataset/Train/Spa/images322 (3).jpg  
  inflating: Dataset/Train/Spa/images322 (4).jpg  
  inflating: Dataset/Train/Spa/images323 (3).jpg  
  inflating: Dataset/Train/Spa/images323.jpg 

In [ ]:
# Mounting my google drive in colab notebook
from google.colab import drive
drive.mount('/content/drive')

In [6]:
# Importing necessary tensorflow libraries
import tensorflow as tf
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3,VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,TopKCategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,TensorBoard

In [7]:
# Image data generator functions
from image_data_gen import train_generator,val_generator

In [8]:
# no. of classes
num_classes=15

In [9]:
# Using InceptionV3 as the base model
base_model=InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)

87916544/87910968 [==============================] - 1s 0us/step


In [12]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 127, 127, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 127, 127, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [10]:
# Creating our layers on top of base model
inputs=Input(shape=(256,256,3))
x=base_model(inputs)
x=GlobalAveragePooling2D()(x)
outputs=Dense(num_classes,activation="softmax")(x)

model=Model(inputs,outputs)

In [11]:
# Summary of the created model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_v3 (Model)         (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 15)                30735     
Total params: 21,833,519
Trainable params: 21,799,087
Non-trainable params: 34,432
_________________________________________________________________


In [13]:
# Adam Optimizer used for training
optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)

In [14]:
# Compiling the model
model.compile(optimizer=optimizer,
                loss=CategoricalCrossentropy(),
                metrics=[CategoricalAccuracy(),
                TopKCategoricalAccuracy(k=3)]
)

In [15]:
# Creating checkpoints
model_checkpoint=ModelCheckpoint(
    filepath="/content/drive/My Drive/HOTEL_IMG_ClASSIFY/model_data_augment_full_train/model_data_augmennt_full_train_checkpoints",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode="min",
    save_freq="epoch"
)

In [16]:
# No. of eppochs
epochs=100

In [ ]:
# Load weights if starting training from a checkpoint

# model.load_weights("/content/drive/My Drive/REAL ESTATE CV/IMG_ClASSIFY/model_data_augment_full_train/model_data_augmennt_full_train_checkpoints/variables/variables")

In [ ]:
# Fitting the model to the data
model.fit(x=train_generator(),
            epochs=epochs,
            steps_per_epoch=72520//32,
            verbose=2,
            callbacks=[model_checkpoint,tensorboard_callback],
            validation_data=val_generator(),
            validation_steps=1475//16,
)

In [17]:
# Evaluating model on weights obtained after full training (also included in repository)
model.load_weights("/content/drive/My Drive/REAL ESTATE CV/IMG_ClASSIFY/model_data_augment_full_train/model_data_augmennt_full_train_checkpoints/variables/variables")

In [18]:
# Evaluating the model

model.evaluate(train_generator())

Found 72520 images belonging to 15 classes.
2267/2267 [==============================] - 1257s 555ms/step - loss: 0.4172 - categorical_accuracy: 0.8564 - top_k_categorical_accuracy: 0.9772


[0.4171983599662781, 0.8564120531082153, 0.9772476553916931]

In [19]:
model.evaluate(val_generator())

Found 1475 images belonging to 15 classes.
93/93 [==============================] - 5s 57ms/step - loss: 0.6946 - categorical_accuracy: 0.7892 - top_k_categorical_accuracy: 0.9539


[0.6945756077766418, 0.7891525626182556, 0.9538983106613159]